In [18]:
# -*- coding: utf-8 -*-
__author__ = 'tyler'
import urllib2
import scrapy
from scrapy import log
import demjson

'''class AutoSpider(scrapy.Spider):
    name = "sse"
    allowed_domains = ["query.sse.com.cn"]
    preurl='http://data.eastmoney.com/stock';
    start_urls = [
        'http://query.sse.com.cn/infodisplay/showTradePublicFile.do?jsonCallBack=jQuery172023210379532913938_1430627585124&dateTx=2015-04-29&random=0.48195114223841695&_=1430627617454'
    ]
    def parse(self, response):
        jsonstr=response.body_as_unicode()
        log.msg(jsonstr[len('jQuery172023210379532913938_1430627585124'):-1])
        s1=demjson.decode(jsonstr[len('jQuery172023210379532913938_1430627585124('):-1])
        log.msg(s1['fileContents'])

if __name__=='__main__':'''


import re
tradeDay=''
send_headers = {
        'Host': 'query.sse.com.cn',
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:37.0) Gecko/20100101 Firefox/37.0',
        'Accept': '*/*',
        'Accept-Language': 'zh-CN,zh;q=0.8,en-US;q=0.5,en;q=0.3',
        'Accept-Encoding': 'gzip, deflate',
        'Referer': 'http://www.sse.com.cn/disclosure/diclosure/public/',
        'Connection': 'keep-alive'
    }
url='http://query.sse.com.cn/infodisplay/showTradePublicFile.do?jsonCallBack=jQuery172023210379532913938_1430627585124&dateTx=2015-04-29&random=0.48195114223841695&_=1430627617454'
req = urllib2.Request(url,headers=send_headers)
response = urllib2.urlopen(req)
html = response.read()
jsonStr=demjson.decode(html[len('jQuery172023210379532913938_1430627585124('):-1])
lines=jsonStr['fileContents']
def loopLineFun(lines):
    for line in lines:
        yield line.encode('utf8')
loopline=loopLineFun(lines)
class LHBItem():
    pass
dictlist = {}
r1 = re.compile(ur'\s+\(\d\)\s+(\d+)\s+([\u4e00-\u9fa5]+)\s+((-?\d+)(\.\d+)?)%\s+(\d+)\s+((-?\d+)(\.\d+)?)')
#r1 = re.compile(ur'\s+\(\d\)')

def readDep(loop,code):
    state='buy'
    rdep = re.compile(ur'\s+\(\d\)')
    rout=re.compile(ur'^\s?$')
    lbhItem=dictlist[code]
    deps=[]
    for tmp in loop:
        print tmp
        if tmp.find('买入营业部名称')>=0:
            state='buy'
            continue
        if tmp.find('卖出营业部名称')>=0:
            state='sell'
            continue
        outMatch=rout.match(tmp)
        if outMatch and state=='sell':
            print '跳出'
            return
        if rdep.match(tmp.decode('utf8')):
            dep=re.split('\s+',tmp)
            depName=dep[2]
            tradeAmount=dep[3]
            if state=='buy':
                deps.append({"depName":depName,"tradeAmount":tradeAmount})
            else:
                deps.append({"depName":depName,"tradeAmount":'-'+tradeAmount})
            print 'depName ' + depName
        lbhItem.deps=deps
        dictlist[lbhItem.symbol]=lbhItem
        
r2=re.compile(ur'\s+[\u4e00-\u9fa5]+:\s(\d+)\s+[\u4e00-\u9fa5]+:\s[\u4e00-\u9fa5]+')
def readA7(loop,outSign):
    for tmp in loop:
        print tmp+'a7'
        mat=r1.match(tmp.decode('utf8'))
        if mat:
            lbhItem =LHBItem()
            lbhItem.symbol= mat.group(1)
            lbhItem.stockName= mat.group(2)
            lbhItem.zhengdie= mat.group(3)
            lbhItem.vol=mat.group(6)
            lbhItem.amount= mat.group(7)
            dictlist[lbhItem.symbol]=lbhItem
            continue
        #dep
        mat2=r2.match(tmp.decode('utf8'))
        if mat2:
            print '*************************'
            readDep(loop,mat2.group(1))
        if tmp.find('2、B股')>=0:
            return


for tmp in loopline:
    print tmp
    if tmp.find('交易日期')>=0:
        tradeDay=tmp[13:]
        print tradeDay
    if tmp.find('偏离值达到7%')>=0:
        tmp=readA7(loopline)
        continue
    if tmp.find('二、有价格涨跌幅限制的日收盘价格跌幅偏离值达到7%的前三只证券')>=0:
        tmp=readA7(loopline)
        continue
    if tmp.find('三、有价格涨跌幅限制的日价格振幅达到15%的前')>=0:
        tmp=readA7(loopline)
        continue
        四、有价格涨跌幅限制的日换手率达到20%的前三只证券:
            五、无价格涨跌幅限制的证券
            六、非ST、*ST和S证券连续三个交易日内收盘价格涨幅偏离值累计达到20%的证券
            七、非ST、*ST和S证券连续三个交易日内收盘价格跌幅偏离值累计达到20%的证券:
            八、ST、*ST和S证券连续三个交易日内收盘价格涨幅偏离值累计达到15%的证券
            
            九、ST、*ST和S证券连续三个交易日内收盘价格跌幅偏离值累计达到15%的证券:
                十、连续三个交易日内的日均换手率与前五个交易日日均换手率的比值到达30倍,并且该股票封闭式基金连续三个交易日内累计换手率达到20%
                十一、当日无价格涨跌幅限制的A股，出现异常波动停牌的:
                    十二、当日无价格涨跌幅限制的B股，出现异常波动停牌的:
                        十三、单只标的证券的当日融资买入数量达到当日该证券总交易量的50％以上:
                            十四、单只标的证券的当日融券卖出数量达到当日该证券总交易量的50％以上:
                                十五、风险警示股票盘中换手率达到或超过30%：
                                十六、退市整理的证券：
                                十八、实施特别停牌的证券:
    if tmp.find('二、')>=0:
        print '-------'
for k in dictlist:
    print k
    



            上海证券交易所每日交易信息

交易日期:2015年04月29日
2015年04月29日


一、有价格涨跌幅限制的日收盘价格涨幅偏离值达到7%的前三只证券:
 1、A股a7
        证券代码      证券简称      偏离值%        成交量        成交金额(万元)a7
    (1)  603318      派思股份      10.03%           16600              20.75a7
    (2)  600311      荣华实业      10.03%       116236335          112940.60a7
    (3)  600839      四川长虹      10.03%       318313807          243244.33a7
a7
      证券代码: 603318                                            证券简称: 派思股份a7
*************************
      ------------------------------------------------------------------------------
      买入营业部名称:                                              累计买入金额(元):
  (1) 招商证券股份有限公司北京知春东里证券营业部                           207500.00
depName 招商证券股份有限公司北京知春东里证券营业部

      卖出营业部名称:                                              累计卖出金额(元):
  (1) 中信证券（浙江）有限责任公司宁波中山东路证券营业部                    12500.00
depName 中信证券（浙江）有限责任公司宁波中山东路证券营业部
  (2) 渤海证券股份有限公司上海定西路证券营业部                              12500.00
depName 渤海证券股份有限公司上海定西路证券营业部


TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'